In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

import seaborn as sns
import statsmodels.api as sm

from longitudinal.settings.constants import DATA_PATH

import warnings
warnings.filterwarnings("ignore")

In [2]:
gen1_train = pd.read_csv(DATA_PATH + "gen1_train_comp_final.csv")
gen2_train = pd.read_csv(DATA_PATH + "gen2_train_comp_final.csv")

gen1_train['Height_Velocity'] = gen1_train.groupby('gen1_id')['SHgt_cm'].diff()
gen2_puberty = gen2_train[(gen2_train['AgeGr'] >= 9) & (gen2_train['AgeGr'] <= 15)].copy()
gen2_puberty['Height_Velocity'] = gen2_puberty.groupby('gen2_id')['SHgt_cm'].diff()
growth_threshold = gen2_puberty['Height_Velocity'].quantile(0.75)

### Puberty Onset Model

In [3]:
puberty_onset = (
    gen2_puberty[gen2_puberty['Height_Velocity'] > growth_threshold]
    .groupby('gen2_id')['AgeGr']
    .min()
    .reset_index()
)
puberty_onset.columns = ['gen2_id', 'Puberty_Onset_Age']


gen1_features = (
    gen1_train
    .groupby('gen1_id')
    .agg(
        Final_Height=('SHgt_cm', 'max'),
        Max_Growth_Rate=('Height_Velocity', 'max'),
        Peak_Growth_Age=('age', lambda x: x.iloc[np.argmax(gen1_train.loc[x.index, 'Height_Velocity'])]),
    ).reset_index()
)
gen1_features.rename(columns={'gen1_id': 'study_parent_id_new'}, inplace=True)

merged_data = (
    puberty_onset
    .merge(gen2_train[['gen2_id', 'study_parent_id_new', 'sex_assigned_at_birth']], on='gen2_id')
    .merge(gen1_features, on='study_parent_id_new')
    .merge(gen1_train[['gen1_id', 'sex_assigned_at_birth']], left_on='study_parent_id_new', right_on='gen1_id', suffixes=('_child', '_parent'))
    .drop(columns=['gen1_id'])
)

merged_data['Sex_Group'] = merged_data['sex_assigned_at_birth_child'] + "_" + merged_data['sex_assigned_at_birth_parent']

formula = """
    Puberty_Onset_Age ~ Final_Height + Max_Growth_Rate + Peak_Growth_Age + 
    C(Sex_Group) + 
    Final_Height:C(Sex_Group) + Max_Growth_Rate:C(Sex_Group) + Peak_Growth_Age:C(Sex_Group)
"""

model = smf.ols(formula, data=merged_data).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      Puberty_Onset_Age   R-squared:                       0.507
Model:                            OLS   Adj. R-squared:                  0.507
Method:                 Least Squares   F-statistic:                     6403.
Date:                Mon, 03 Mar 2025   Prob (F-statistic):               0.00
Time:                        20:41:16   Log-Likelihood:            -1.2945e+05
No. Observations:               87120   AIC:                         2.589e+05
Df Residuals:                   87105   BIC:                         2.591e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               8.7466      0.294     29.716      0.000       8.170       9.324
C(Sex_Group)[T.F_M]                     3.1895      0.376      8.493      0.000       2.453       3.926
C(Sex_Group)[T.M_F]                    -0.2726      0.037     -7.329      0.000      -0.345      -0.200
C(Sex_Group)[T.M_M]                     2.5955      0.365      7.120      0.000       1.881       3.310
Final_Height                            0.0448      0.002     25.676      0.000       0.041       0.048
Final_Height:C(Sex_Group)[T.F_M]       -0.0467      0.002    -20.877      0.000      -0.051      -0.042
Final_Height:C(Sex_Group)[T.M_F]       -0.0045      0.002     -2.122      0.034      -0.009      -0.000
Final_Height:C(Sex_Group)[T.M_M]       -0.0377      0.002    -18.608      0.000      -0.042      -0.034
Max_Growth_Rate                        -0.4310      0.006    -67.226      0.000      -0.444      -0.418
Max_Growth_Rate:C(Sex_Group)[T.F_M]     0.3829      0.010     38.476      0.000       0.363       0.402
Max_Growth_Rate:C(Sex_Group)[T.M_F]     0.5921      0.009     63.189      0.000       0.574       0.610
Max_Growth_Rate:C(Sex_Group)[T.M_M]     0.4566      0.009     48.870      0.000       0.438       0.475
Peak_Growth_Age                        -0.1763      0.006    -27.730      0.000      -0.189      -0.164
Peak_Growth_Age:C(Sex_Group)[T.F_M]     0.2171      0.011     18.991      0.000       0.195       0.240
Peak_Growth_Age:C(Sex_Group)[T.M_F]    -0.8177      0.112     -7.329      0.000      -1.036      -0.599
Peak_Growth_Age:C(Sex_Group)[T.M_M]     0.3075      0.027     11.489      0.000       0.255       0.360
==============================================================================
Omnibus:                      474.817   Durbin-Watson:                   0.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              482.117
Skew:                          -0.181   Prob(JB):                    2.04e-105
Kurtosis:                       2.958   Cond. No.                     3.98e+15
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.04e-22. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

### Puberty Duration Model

In [4]:
puberty_duration = (
    gen2_puberty[gen2_puberty['Height_Velocity'] > growth_threshold]
    .groupby('gen2_id')['AgeGr']
    .count()
    .reset_index()
)
puberty_duration.columns = ['gen2_id', 'Puberty_Duration']

gen2_puberty_info = puberty_onset.merge(puberty_duration, on='gen2_id')


merged_data = (
    gen2_puberty_info
    .merge(gen2_train[['gen2_id', 'study_parent_id_new', 'sex_assigned_at_birth']], on='gen2_id')
    .merge(gen1_features, on='study_parent_id_new')
    .merge(gen1_train[['gen1_id', 'sex_assigned_at_birth']], left_on='study_parent_id_new', right_on='gen1_id', suffixes=('_child', '_parent'))
    .drop(columns=['gen1_id'])
)

merged_data['Sex_Group'] = merged_data['sex_assigned_at_birth_child'] + "_" + merged_data['sex_assigned_at_birth_parent']

formula_duration = """
    Puberty_Duration ~ Final_Height + Max_Growth_Rate + Peak_Growth_Age + 
    C(Sex_Group) + 
    Final_Height:C(Sex_Group) + Max_Growth_Rate:C(Sex_Group) + Peak_Growth_Age:C(Sex_Group)
"""

model_duration = smf.ols(formula_duration, data=merged_data).fit()
model_duration.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Puberty_Duration   R-squared:                       0.088
Model:                            OLS   Adj. R-squared:                  0.088
Method:                 Least Squares   F-statistic:                     601.6
Date:                Mon, 03 Mar 2025   Prob (F-statistic):               0.00
Time:                        20:41:17   Log-Likelihood:                -91036.
No. Observations:               87120   AIC:                         1.821e+05
Df Residuals:                   87105   BIC:                         1.822e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               2.0315      0.189     10.726      0.000       1.660       2.403
C(Sex_Group)[T.F_M]                    -4.1993      0.242    -17.378      0.000      -4.673      -3.726
C(Sex_Group)[T.M_F]                     0.5832      0.024     24.368      0.000       0.536       0.630
C(Sex_Group)[T.M_M]                    -0.1054      0.235     -0.449      0.653      -0.565       0.354
Final_Height                           -0.0136      0.001    -12.114      0.000      -0.016      -0.011
Final_Height:C(Sex_Group)[T.F_M]        0.0338      0.001     23.487      0.000       0.031       0.037
Final_Height:C(Sex_Group)[T.M_F]       -0.0236      0.001    -17.149      0.000      -0.026      -0.021
Final_Height:C(Sex_Group)[T.M_M]        0.0087      0.001      6.648      0.000       0.006       0.011
Max_Growth_Rate                         0.1589      0.004     38.513      0.000       0.151       0.167
Max_Growth_Rate:C(Sex_Group)[T.F_M]    -0.1424      0.006    -22.229      0.000      -0.155      -0.130
Max_Growth_Rate:C(Sex_Group)[T.M_F]    -0.1753      0.006    -29.080      0.000      -0.187      -0.164
Max_Growth_Rate:C(Sex_Group)[T.M_M]    -0.1270      0.006    -21.125      0.000      -0.139      -0.115
Peak_Growth_Age                         0.1199      0.004     29.301      0.000       0.112       0.128
Peak_Growth_Age:C(Sex_Group)[T.F_M]    -0.1318      0.007    -17.917      0.000      -0.146      -0.117
Peak_Growth_Age:C(Sex_Group)[T.M_F]     1.7495      0.072     24.368      0.000       1.609       1.890
Peak_Growth_Age:C(Sex_Group)[T.M_M]     0.0501      0.017      2.907      0.004       0.016       0.084
==============================================================================
Omnibus:                     3432.859   Durbin-Watson:                   0.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3669.325
Skew:                           0.483   Prob(JB):                         0.00
Kurtosis:                       2.724   Cond. No.                     3.98e+15
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.04e-22. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

### Total Pubertal Growth Model

In [5]:
puberty_growth = (
    gen2_puberty
    .groupby('gen2_id')
    .agg(
        Min_Height=("SHgt_cm", "min"),
        Max_Height=("SHgt_cm", "max")
    )
    .reset_index()
)
puberty_growth["Child_Growth"] = puberty_growth["Max_Height"] - puberty_growth["Min_Height"]
puberty_growth.drop(columns=["Min_Height", "Max_Height"], inplace=True)
merged_data = merged_data.merge(puberty_growth, on=["gen2_id"], how="left")

formula_growth = """
    Child_Growth ~ Final_Height + Max_Growth_Rate + Peak_Growth_Age + 
    C(Sex_Group) + 
    Final_Height:C(Sex_Group) + Max_Growth_Rate:C(Sex_Group) + Peak_Growth_Age:C(Sex_Group)
"""

model_duration = smf.ols(formula_growth, data=merged_data).fit()
model_duration.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Child_Growth   R-squared:                       0.448
Model:                            OLS   Adj. R-squared:                  0.447
Method:                 Least Squares   F-statistic:                     5040.
Date:                Mon, 03 Mar 2025   Prob (F-statistic):               0.00
Time:                        20:41:17   Log-Likelihood:            -2.4495e+05
No. Observations:               87120   AIC:                         4.899e+05
Df Residuals:                   87105   BIC:                         4.901e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                             -11.7216      1.108    -10.577      0.000     -13.894      -9.550
C(Sex_Group)[T.F_M]                     1.2064      1.414      0.853      0.394      -1.565       3.978
C(Sex_Group)[T.M_F]                     6.4910      0.140     46.355      0.000       6.217       6.765
C(Sex_Group)[T.M_M]                     2.4664      1.373      1.797      0.072      -0.224       5.157
Final_Height                            0.3080      0.007     46.861      0.000       0.295       0.321
Final_Height:C(Sex_Group)[T.F_M]       -0.0350      0.008     -4.163      0.000      -0.052      -0.019
Final_Height:C(Sex_Group)[T.M_F]       -0.3627      0.008    -44.977      0.000      -0.378      -0.347
Final_Height:C(Sex_Group)[T.M_M]       -0.1013      0.008    -13.288      0.000      -0.116      -0.086
Max_Growth_Rate                        -1.0486      0.024    -43.442      0.000      -1.096      -1.001
Max_Growth_Rate:C(Sex_Group)[T.F_M]     0.3053      0.037      8.149      0.000       0.232       0.379
Max_Growth_Rate:C(Sex_Group)[T.M_F]     0.4133      0.035     11.715      0.000       0.344       0.482
Max_Growth_Rate:C(Sex_Group)[T.M_M]     0.9871      0.035     28.061      0.000       0.918       1.056
Peak_Growth_Age                        -0.0174      0.024     -0.727      0.467      -0.064       0.030
Peak_Growth_Age:C(Sex_Group)[T.F_M]    -0.6930      0.043    -16.099      0.000      -0.777      -0.609
Peak_Growth_Age:C(Sex_Group)[T.M_F]    19.4729      0.420     46.355      0.000      18.650      20.296
Peak_Growth_Age:C(Sex_Group)[T.M_M]     2.7799      0.101     27.587      0.000       2.582       2.977
==============================================================================
Omnibus:                     3586.242   Durbin-Watson:                   0.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4750.184
Skew:                          -0.430   Prob(JB):                         0.00
Kurtosis:                       3.755   Cond. No.                     3.98e+15
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.04e-22. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""